In [1]:
import pandas as pd
from koreanize_matplotlib import koreanize
koreanize()

In [2]:
df = pd.read_csv('./data/수산물_통합전처리_3컬럼.csv')

In [3]:
sites = ['여수', '제주도', '안흥', '군산', '통영']
df = df[df['산지'].isin(sites)]
df
# 산지 중 '여수', '제주도', '안흥', '군산', '통영'만 가지고 오면 17695개의 행이 가져와짐

,어종,산지,규격,포장,수량,중량,낙찰고가,낙찰저가,평균가,파일어종,year,month,date,전처리,품목명,공통어종
11,(냉)갈치,제주도,중,CT/(BT),250,10.0,"106,000","106,000","106,000",갈치,2021,1,2021-01-01,냉,갈치,갈치
14,(냉)갈치,제주도,40/45미,CT/(BT),550,10.0,"75,000","75,000","75,000",갈치,2021,1,2021-01-01,냉,갈치,갈치
16,(냉)갈치,제주도,대,CT/(BT),300,10.0,"146,000","146,000","146,000",갈치,2021,1,2021-01-01,냉,갈치,갈치
17,(선)은갈치,제주도,4미,S/P,16,5.5,"175,000","140,000","146,600",갈치,2021,1,2021-01-01,선,은갈치,갈치
18,(선)은갈치,제주도,10미,S/P,14,4.3,"85,000","85,000","85,000",갈치,2021,1,2021-01-01,선,은갈치,갈치
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37546,(선)홍어,군산,10미,S/P,4,15.0,"78,000","78,000","78,000",홍어,2024,12,2024-12-01,선,홍어,홍어
37547,(선)홍어,안흥,2/3미,S/P,91.8,1.0,"8,000","7,000","7,600",홍어,2024,12,2024-12-01,선,홍어,홍어
37548,(선)홍어,안흥,2미,S/P,1,23.0,"138,000","138,000","138,000",홍어,2024,12,2024-12-01,선,홍어,홍어
37552,(선)홍어,안흥,5/6미,kg,638.3,1.0,"3,500","3,000","3,100",홍어,2024,12,2024-12-01,선,홍어,홍어


In [4]:
# 산지 '여수', '제주도', '안흥', '군산', '통영' 기준으로 필터링한 df 와 해양데이터 합치기


sanji_df = pd.read_csv('./data/해양정보_추출/산지별_2021_2024_해양데이터.csv')    # 원본 해양데이터 데이터프레임

# year, month, 산지 기준으로 병합
merged_df = pd.merge(df, sanji_df, on=['year', 'month', '산지'], how='inner')


In [26]:
merged_df.head(2)

,어종,산지,규격,포장,수량,중량,낙찰고가,낙찰저가,평균가,파일어종,year,month,date,전처리,품목명,공통어종,기온 평균,수온 평균,풍속 평균
0,(냉)갈치,제주도,중,CT/(BT),250,10.0,"106,000","106,000","106,000",갈치,2021,1,2021-01-01,냉,갈치,갈치,7.2,14.4,5.0
1,(냉)갈치,제주도,40/45미,CT/(BT),550,10.0,"75,000","75,000","75,000",갈치,2021,1,2021-01-01,냉,갈치,갈치,7.2,14.4,5.0


In [5]:
# 빈값 추출
merged_df.isna().sum()
# 결측치가 하나라도 있는 행을 삭제하고, 결과를 원본에 저장
merged_df.dropna(axis=0, how='any', inplace=True)

# 삭제 후 남은 데이터 개수 확인
merged_df.isna().sum()

어종       0
산지       0
규격       0
포장       0
수량       0
중량       0
낙찰고가     0
낙찰저가     0
평균가      0
파일어종     0
year     0
month    0
date     0
전처리      0
품목명      0
공통어종     0
기온 평균    0
수온 평균    0
풍속 평균    0
dtype: int64

In [6]:
merged_df['어종'].value_counts().head(2)

어종
(선)은갈치    7154
(선)참조기    1523
Name: count, dtype: int64

In [6]:
print(merged_df.dtypes)

어종        object
산지        object
규격        object
포장        object
수량        object
중량       float64
낙찰고가      object
낙찰저가      object
평균가       object
파일어종      object
year       int64
month      int64
date      object
전처리       object
품목명       object
공통어종      object
기온 평균    float64
수온 평균    float64
풍속 평균    float64
dtype: object


In [7]:
cols_to_int = ['수량', '낙찰고가', '낙찰저가', '평균가']

for col in cols_to_int:
    # 쉼표 모두 제거 (문자열로 가정)
    merged_df[col] = merged_df[col].str.replace(',', '', regex=True)
    # 숫자형으로 변환(변환 불가면 NaN 처리)
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')
    # NaN은 0으로 대체 후 int형 변환
    merged_df[col] = merged_df[col].fillna(0).astype(int)


In [8]:
merged_df.isna().sum()

어종       0
산지       0
규격       0
포장       0
수량       0
중량       0
낙찰고가     0
낙찰저가     0
평균가      0
파일어종     0
year     0
month    0
date     0
전처리      0
품목명      0
공통어종     0
기온 평균    0
수온 평균    0
풍속 평균    0
dtype: int64

In [18]:
# '어종'이 '(선)은갈치'인 행만 필터링
merged_df[merged_df['어종'] == '(선)은갈치']

,어종,산지,규격,포장,수량,중량,낙찰고가,낙찰저가,평균가,파일어종,year,month,date,전처리,품목명,공통어종,기온 평균,수온 평균,풍속 평균
3,(선)은갈치,제주도,4미,S/P,16,5.5,"175,000","140,000","146,600",갈치,2021,1,2021-01-01,선,은갈치,갈치,7.2,14.4,5.0
4,(선)은갈치,제주도,10미,S/P,14,4.3,"85,000","85,000","85,000",갈치,2021,1,2021-01-01,선,은갈치,갈치,7.2,14.4,5.0
5,(선)은갈치,제주도,4미,S/P,1,2.5,"45,000","45,000","45,000",갈치,2021,1,2021-01-01,선,은갈치,갈치,7.2,14.4,5.0
6,(선)은갈치,제주도,4미,S/P,1,4.0,"150,000","150,000","150,000",갈치,2021,1,2021-01-01,선,은갈치,갈치,7.2,14.4,5.0
7,(선)은갈치,제주도,20미,S/P,281,5.0,"60,000","37,000","50,400",갈치,2021,1,2021-01-01,선,은갈치,갈치,7.2,14.4,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8622,(선)은갈치,제주도,7미,S/P,2,3.0,"60,000","40,000","50,000",갈치,2024,12,2024-12-01,선,은갈치,갈치,9.3,17.2,4.8
8623,(선)은갈치,제주도,3미,S/P,1,2.3,"55,000","55,000","55,000",갈치,2024,12,2024-12-01,선,은갈치,갈치,9.3,17.2,4.8
8624,(선)은갈치,제주도,6미,S/P,1,3.7,"100,000","100,000","100,000",갈치,2024,12,2024-12-01,선,은갈치,갈치,9.3,17.2,4.8
8625,(선)은갈치,제주도,27미,S/P,1,10.0,"60,000","60,000","60,000",갈치,2024,12,2024-12-01,선,은갈치,갈치,9.3,17.2,4.8


In [25]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# 어종과 산지 고유값 리스트, 'all' 포함
fish_list = ['all'] + merged_df['어종'].unique().tolist()
area_list = ['all'] + merged_df['산지'].unique().tolist()

# 드롭다운 생성
fish_dropdown = widgets.Dropdown(options=fish_list, value='all', description='어종 선택:')
area_dropdown = widgets.Dropdown(options=area_list, value='all', description='산지 선택:')

output = widgets.Output()

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected_fish = fish_dropdown.value
        selected_area = area_dropdown.value
        
        clear_output(wait=True)  # 이전 출력 초기화
        display(fish_dropdown, area_dropdown, output)  # UI와 출력 영역 재표시
        
        with output:
            output.clear_output()
            # 둘 다 'all'이 아니면 실행
            if selected_fish != 'all' and selected_area != 'all':
                df_filtered = merged_df[
                    (merged_df['어종'] == selected_fish) &
                    (merged_df['산지'] == selected_area)
                ]
                cols = ['수량', '중량', '낙찰고가', '낙찰저가', '평균가', '기온 평균', '수온 평균', '풍속 평균']

                # 쉼표 제거 후 숫자형 변환
                # for col in cols:
                #     if col in ['낙찰고가', '낙찰저가', '평균가']:
                #         df_filtered[col] = df_filtered[col].str.replace(',', '', regex=True)
                #     df_filtered[col] = pd.to_numeric(df_filtered[col], errors='coerce')

                df_filtered = df_filtered.dropna(subset=cols)

                scaler = StandardScaler()
                scaled_values = scaler.fit_transform(df_filtered[cols])
                df_scaled = pd.DataFrame(scaled_values, columns=cols, index=df_filtered.index)

                corr = df_scaled.corr()
                print(f"어종: {selected_fish}, 산지: {selected_area} 상관계수 (스케일링 후):")
                display(corr)

                plt.figure(figsize=(10, 8))
                sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
                plt.title(f"어종: {selected_fish}, 산지: {selected_area} 상관관계 히트맵")
                plt.show()

                sns.pairplot(df_scaled)
                plt.suptitle(f"어종: {selected_fish}, 산지: {selected_area} 산점도 행렬", y=1.02)
                plt.show()
            else:
                print("어종과 산지를 모두 선택해주세요.")

# 두 드롭다운 모두 observe 함수와 연결
fish_dropdown.observe(on_change)
area_dropdown.observe(on_change)

# 화면에 드롭다운과 출력 영역 표시 (초기 UI 출력)
display(fish_dropdown, area_dropdown, output)


Dropdown(description='어종 선택:', options=('all', '(냉)갈치', '(선)은갈치', '(냉)은갈치', '(선)갈치 기타', '(선)갈치', '(선)먹갈치', '(냉…

Dropdown(description='산지 선택:', options=('all', '제주도', '통영', '군산', '안흥', '여수'), value='all')

Output()

In [17]:
merged_df[merged_df['어종'] == '(선)갈치']['산지'].unique()

array(['통영', '군산', '제주도', '안흥'], dtype=object)

In [24]:
merged_df

,어종,산지,규격,포장,수량,중량,낙찰고가,낙찰저가,평균가,파일어종,year,month,date,전처리,품목명,공통어종,기온 평균,수온 평균,풍속 평균
0,(냉)갈치,제주도,중,CT/(BT),250,10.0,106000,106000,106000,갈치,2021,1,2021-01-01,냉,갈치,갈치,7.2,14.4,5.0
1,(냉)갈치,제주도,40/45미,CT/(BT),550,10.0,75000,75000,75000,갈치,2021,1,2021-01-01,냉,갈치,갈치,7.2,14.4,5.0
2,(냉)갈치,제주도,대,CT/(BT),300,10.0,146000,146000,146000,갈치,2021,1,2021-01-01,냉,갈치,갈치,7.2,14.4,5.0
3,(선)은갈치,제주도,4미,S/P,16,5.5,175000,140000,146600,갈치,2021,1,2021-01-01,선,은갈치,갈치,7.2,14.4,5.0
4,(선)은갈치,제주도,10미,S/P,14,4.3,85000,85000,85000,갈치,2021,1,2021-01-01,선,은갈치,갈치,7.2,14.4,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17690,(선)홍어,군산,10미,S/P,4,15.0,78000,78000,78000,홍어,2024,12,2024-12-01,선,홍어,홍어,4.1,8.7,4.6
17691,(선)홍어,안흥,2/3미,S/P,91,1.0,8000,7000,7600,홍어,2024,12,2024-12-01,선,홍어,홍어,4.9,12.5,4.8
17692,(선)홍어,안흥,2미,S/P,1,23.0,138000,138000,138000,홍어,2024,12,2024-12-01,선,홍어,홍어,4.9,12.5,4.8
17693,(선)홍어,안흥,5/6미,kg,638,1.0,3500,3000,3100,홍어,2024,12,2024-12-01,선,홍어,홍어,4.9,12.5,4.8
